# Final Project: Decide on global dual sourcing strategy

team: \
David Yang \
Jack Chen \
Joyce Wu

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('final project 2024.csv')

In [3]:
data.head(10)

,period,demand
0,1,21
1,2,81
2,3,32
3,4,58
4,5,47
5,6,49
6,7,66
7,8,29
8,9,55
9,10,39


In [4]:
s = 10 # sales price
c_m = 8 # cost mexico
c_c = 7.25 # cost china
i = 0.01 # interest rate
ini_bal = 0 # initial balance

## Single Sourcing: Mexico

### Cumulative Average Forecast

In [10]:
def single_source_mexico_cum_avg(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance):
    cash_balance = initial_cash_balance
    inventory = 0
    
    for period in data['period']:
        current_demand = data.loc[data['period'] == period, 'demand'].values[0]
        
        # Calculate cumulative average demand up to the current period
        if period > 1:
            cum_avg_demand = np.ceil(np.mean(data.loc[data['period'] < period, 'demand']))
        else:
            cum_avg_demand = np.ceil(current_demand)  # For the first period, use the current demand
        
        beginning_inventory = inventory
        
        # Satisfy demand from inventory
        if inventory >= current_demand:
            sales = current_demand
            inventory -= sales
        else:
            sales = inventory
            inventory = 0
        
        # Calculate revenue
        revenue = sales * sales_price
        cash_balance += revenue

        # Determine order quantity based
        if cum_avg_demand >= inventory:
            order_quantity = int(cum_avg_demand - inventory)
        else:
            order_quantity = 0
        
        order_cost = order_quantity * sourcing_cost
        cash_balance -= order_cost
        current_cash = cash_balance # record current balance

        # Apply interest
        cash_balance = cash_balance * (1 + interest_rate)

        # Record ending inventory before the new order arrives
        ending_inventory = inventory

        # Update inventory with new order
        inventory += order_quantity
        
        # Debugging outputs
        """
        print(f"Period: {period}")
        print(f"Beginning Inventory: {beginning_inventory}")
        print(f"Current Demand: {current_demand}")
        print(f"Sales: {sales}")
        print(f"Revenue: {revenue}")
        print(f"Order Quantity: {order_quantity}")
        print(f"Order Cost: {order_cost}")
        print(f"Ending Inventory: {ending_inventory}")
        print(f"Profit from Operation: {revenue - order_cost}")
        print(f"Profit from Interest: {cash_balance - current_cash}")
        print(f"Cash Balance: {cash_balance}")
        print("-" * 40)
        """
    
    return cash_balance

# Calculate ending bank account value
ending_bank_account_value_1 = single_source_mexico_cum_avg(data, s, c_m, i, ini_bal)

### Moving Average Forecast

In [23]:
def single_source_mexico_mov_avg(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, moving_avg_window, percentile_threshold):
    cash_balance = initial_cash_balance
    inventory = 0
    errors = []
    
    for period in data['period']:
        current_demand = data.loc[data['period'] == period, 'demand'].values[0]

        # Calculate moving average demand over the specified window
        if period > moving_avg_window:
            moving_avg_demand = np.ceil(np.mean(data.loc[(data['period'] >= period - moving_avg_window) & (data['period'] < period), 'demand']))
        else:
            moving_avg_demand = np.ceil(np.mean(data.loc[data['period'] <= period, 'demand']))

        beginning_inventory = inventory
        
        # Satisfy demand from inventory
        if inventory >= current_demand:
            sales = current_demand
            inventory -= sales
        else:
            sales = inventory
            inventory = 0
        
        # Calculate revenue
        revenue = sales * sales_price
        cash_balance += revenue
        
        # Determine order quantity based on moving average demand and current inventory
        order_quantity = 0
        order_cost = 0
        if period > moving_avg_window:
            # Calculate error and update errors list
            forecast_error = current_demand - moving_avg_demand
            errors.append(forecast_error)
            
            # Calculate the 95th percentile of errors
            if len(errors) > 0:
                error_percentile = np.percentile(errors, percentile_threshold)
            else:
                error_percentile = 0

            # Adjust order quantity based on forecast and error percentile
            adjusted_forecast = moving_avg_demand + error_percentile
            if adjusted_forecast >= inventory:
                order_quantity = int(adjusted_forecast - inventory)
            else:
                order_quantity = 0

            order_cost = order_quantity * sourcing_cost
            cash_balance -= order_cost
            current_cash = cash_balance # record current balance
            
            ending_inventory = inventory

            # Update inventory with new order
            inventory += order_quantity
        else:
            ending_inventory = inventory

        # Apply interest
        cash_balance = cash_balance * (1 + interest_rate)

        '''
        print(f"Period: {period}")
        print(f"Beginning Inventory: {beginning_inventory}")
        print(f"Current Demand: {current_demand}")
        print(f"Sales: {sales}")
        print(f"Revenue: {revenue}")
        print(f"Order Quantity: {order_quantity}")
        print(f"Order Cost: {order_cost}")
        print(f"Ending Inventory: {ending_inventory}")
        print(f"Profit from Operation: {revenue - order_cost}")
        print(f"Profit from Interest: {cash_balance - current_cash}")
        print(f"Cash Balance: {cash_balance}")
        print(f"Forecast Error: {forecast_error}")
        print(f"95th Percentile of Errors: {error_percentile}")
        print(f"Adjusted Forecast: {adjusted_forecast}")
        print("-" * 40)
        '''
    
    return cash_balance

# Calculate ending bank account value with specified moving average window
window = 10
SL = 95
ending_bank_account_value_2 = single_source_mexico_mov_avg(data, s, c_m, i, ini_bal, window, SL)

### Exponential Smoothing

In [26]:
def single_source_mexico_exp_smt(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, alpha, percentile_threshold):
    cash_balance = initial_cash_balance
    inventory = 0
    errors = []
    forecast = None 
    
    for period in data['period']:
        current_demand = data.loc[data['period'] == period, 'demand'].values[0]
        
        # Calculate exponential smoothing forecast
        if period == 1:
            forecast = current_demand  # Use the first period's demand as the initial forecast
        else:
            forecast = alpha * current_demand + (1 - alpha) * forecast

        # Calculate forecast error and update errors list
        if period > 1:
            forecast_error = current_demand - forecast
            errors.append(forecast_error)
            
            # Calculate the 95th percentile of errors
            if len(errors) > 0:
                error_percentile = np.percentile(errors, percentile_threshold)
            else:
                error_percentile = 0

            # Adjust order quantity based on forecast and error percentile
            adjusted_forecast = forecast + error_percentile
            if adjusted_forecast >= inventory:
                order_quantity = int(adjusted_forecast - inventory)
            else:
                order_quantity = 0

            order_cost = order_quantity * sourcing_cost
            cash_balance -= order_cost
            current_cash = cash_balance  # Record current balance
            
            # Record ending inventory before the new order arrives
            ending_inventory = inventory

            # Update inventory with new order
            inventory += order_quantity
        else:
            ending_inventory = inventory

        # Satisfy demand from inventory
        if inventory >= current_demand:
            sales = current_demand
            inventory -= sales
        else:
            sales = inventory
            inventory = 0
        
        # Calculate revenue
        revenue = sales * sales_price
        cash_balance += revenue

        # Apply interest
        cash_balance = cash_balance * (1 + interest_rate)

        '''
        print(f"Period: {period}")
        print(f"Beginning Inventory: {beginning_inventory}")
        print(f"Current Demand: {current_demand}")
        print(f"Sales: {sales}")
        print(f"Revenue: {revenue}")
        print(f"Order Quantity: {order_quantity}")
        print(f"Order Cost: {order_cost}")
        print(f"Ending Inventory: {ending_inventory}")
        print(f"Profit from Operation: {revenue - order_cost}")
        print(f"Profit from Interest: {cash_balance - current_cash}")
        print(f"Cash Balance: {cash_balance}")
        print(f"Forecast Error: {forecast_error}")
        print(f"95th Percentile of Errors: {error_percentile}")
        print(f"Adjusted Forecast: {adjusted_forecast}")
        print("-" * 40)
        '''
    
    return cash_balance

# Optimization function to find the best alpha
def optimize_alpha(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, percentile_threshold, alpha_values):
    best_alpha = None
    best_value = -np.inf
    
    for alpha in alpha_values:
        ending_bank_account_value = single_source_mexico_exp_smt(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, alpha, percentile_threshold)
        if ending_bank_account_value > best_value:
            best_value = ending_bank_account_value
            best_alpha = alpha
    
    return best_alpha, best_value

# Define range of alpha values to test
alpha_values = np.linspace(0.01, 0.99, 99)
SL = 95
# Optimize alpha
best_alpha, best_value = optimize_alpha(data, s, c_m, i, ini_bal, SL, alpha_values)
print(f"Best Alpha: {best_alpha:.2f}")

Best Alpha: 0.99


### Results

In [28]:
print("Cumulative Average Forecast: ${:,.2e}".format(ending_bank_account_value_1))
print("10-Day Moving Average Forecast (95% Service Level): ${:,.2e}".format(ending_bank_account_value_2))
print(f"Exponential Smoothing Forecast (95% Service Level): ${best_value:,.2e}")

Cumulative Average Forecast: $1.24e+47
10-Day Moving Average Forecast (95% Service Level): $1.28e+47
Exponential Smoothing Forecast (95% Service Level): $1.55e+47


## Single Sourcing: China

## Duo Sourcing Strategy